In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
import time
from shapely import wkt
import datetime
from shapely.geometry import Point
font = {'family' : 'Sans',
        'size'   : 15}
matplotlib.rc('font', **font)

/Users/ninawiedemann/miniconda3/envs/gis/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Helper functions

In [3]:
convert_to_timestamp = lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f").timetuple())

def lon_lat_to_geom(data, lon_name="LON", lat_name="LAT"):
    geom_col = []
    for i, row in data.iterrows():
        geom = Point(row[lon_name], row[lat_name])
        geom_col.append(geom)
    data["geom"] = geom_col
    data = gpd.GeoDataFrame(data, geometry="geom")
    return data

def write_geodataframe(gdf, out_path):
    geo_col_name = gdf.geometry.name
    df = pd.DataFrame(gdf, copy=True)
    df[geo_col_name] = gdf.geometry.apply(wkt.dumps)
    df.to_csv(out_path, index=True)
    
def read_geodataframe(in_path, geom_col="geom"):
    df = pd.read_csv(in_path)
    df[geom_col] = df[geom_col].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry=geom_col)
    return gdf

# README

This script is partly exploration and partly data cleaning, and the last part of this notebook was used to write the data to our PostgreSQL database.
In an intermediate step, the data was saved into a new folder called `data_cleaned`. This folder is created in the following cell.

Changes to the data:
* Merge monthly csvs of reservations & drop duplicates
* Some columns are added that we thought might come in handy later, e.g. the reservation duration.
* Convert LON LAT columns to a geopandas geometry and save as a Geodataframe using WKT

In [ ]:
os.makedirs("data_cleaned")

## USER

In [ ]:
data_user = pd.read_csv("data/20211213_ethz_person.tsv", sep="\t")
print("Number users", len(data_user))

In [ ]:
data_user

In [ ]:
assert len(np.unique(data_user["PERSON_NO"])) == len(data_user)
for col in ["AGEGROUP", "GENDER", "LANGUAGE", "ABOGROUP"]:
    sns.countplot(data=data_user, x=col)
    if len(np.unique(data_user[col].dropna())) > 4:
        plt.xticks(rotation=90)
    plt.show()

In [ ]:
# add geometry
geo_frame = lon_lat_to_geom(data_user)

In [ ]:
# add agegroup as an integer
age_group = data_user["AGEGROUP"].values
age_group = age_group[~pd.isna(age_group)]
sorted_groups = np.sort(np.unique(age_group))
map_dict = {g: i for i, g in enumerate(sorted_groups)}
geo_frame["AGEGROUP_int"] = geo_frame["AGEGROUP"].apply(lambda x: map_dict.get(x, pd.NA))

In [ ]:
# convert time to timestamp
geo_frame["FIRSTENTRY_TIMESTAMP"] = geo_frame["FIRSTENTRYDATE"].apply(convert_to_timestamp)

In [ ]:
assert len(np.unique(geo_frame["PERSON_NO"])) == len(data_user)

#### WRITE (and read for test)

In [ ]:
write_geodataframe(geo_frame.set_index("PERSON_NO"), "data_cleaned/user.csv")

In [ ]:
test = read_geodataframe("data_cleaned/user.csv").set_index("PERSON_NO")

## VEHICLE

In [ ]:
data_cars = pd.read_csv("data/20211213_ethz_vehicle.tsv", sep="\t")
# assert len(np.unique(data_cars["VEHICLE_NO"])) == len(data_cars) 
# assert wrong -> maybe index is actual vehicle number?? the other one is the type?
dropped = data_cars.drop_duplicates()
len(data_cars), len(dropped), len(np.unique(data_cars["VEHICLE_NO"])), len(data_cars.dropna())

In [ ]:
data_cars

#### after removing duplicates, add the count to each row

In [ ]:
vehicle_count = data_cars.groupby("VEHICLE_NO").agg({"VEHICLE_NO":"count"})
vehicle_count.rename(columns={"VEHICLE_NO": "COUNT"}, inplace=True)
data_cars_cleaned = dropped.set_index("VEHICLE_NO").merge(vehicle_count, left_index=True, right_index=True)
assert np.sum(data_cars_cleaned["COUNT"]) == len(data_cars)

#### WRITE data

In [ ]:
data_cars_cleaned.to_csv("data_cleaned/vehicle.csv", index="VEHICLE_NO")

In [ ]:
test = pd.read_csv("data_cleaned/vehicle.csv", index_col="VEHICLE_NO")
test

#### Exploration

In [ ]:
for col in ["VEHICLE_CATEGORY", "BRAND_NAME", "ENERGYTYPEGROUP", "ENERGYTYPE"]:
    sns.countplot(data=data_cars, x=col)
    if len(np.unique(data_cars[col].dropna())) > 4:
        plt.xticks(rotation=90)
    plt.show()

## Booking data

### LOAD ALL DATA
(They are split into months, but we merge them here)

In [ ]:
booking_path = "data/20211213_ethz_reservation"
all_bookings = []
for booking_csv in sorted(os.listdir(booking_path)):
    next_csv = pd.read_csv(os.path.join(booking_path, booking_csv), sep="\t")
    all_bookings.append(next_csv)
all_bookings = pd.concat(all_bookings)

In [ ]:
len(all_bookings)

In [ ]:
sorted(os.listdir(booking_path))

In [ ]:
data_booking = all_bookings.drop_duplicates()

In [ ]:
len(data_booking)

### Exploration (bookings)

In [ ]:
duration_sec = (data_booking["RESERVATIONTO"].apply(convert_to_timestamp) - data_booking["RESERVATIONFROM"].apply(convert_to_timestamp)) / 3600


In [ ]:
plt.figure(figsize=(20,10))
plt.hist(duration_sec[duration_sec<100], bins=100)
plt.xlim(0, 100)
plt.yscale("log")
plt.xlabel("Reservation duration (in hours)")
plt.show()

### How spontaneous are the booking

In [ ]:
data_booking[["SYSCREATEDATE","RESERVATIONFROM", "RESERVATIONTO"]]

In [ ]:
sys_create_timestamp = data_booking["SYSCREATEDATE"].apply(convert_to_timestamp)
reservation_from_timestamp = data_booking["RESERVATIONFROM"].apply(convert_to_timestamp)

In [ ]:
spontaneous = (sys_create_timestamp - reservation_from_timestamp) / 3600


In [ ]:
# how many are less than 0.5 hours before reservation
print(f"{round(sum(spontaneous > -0.5) / len(spontaneous)*100,2)}% of bookings are less than half an our before the reservation period")


In [ ]:
data_cut = spontaneous[spontaneous>-300]
plt.figure(figsize=(20,10))
plt.hist(data_cut[data_cut<2], bins=100)
plt.xlim(-300, 10)
# plt.yscale("log")
plt.xlabel("How Spontaneous (in hours)")
plt.show()

#### General plot column distributions again

In [ ]:
for col in ["RESERVATIONTYPE", "RESERVATIONSTATE", "TRIPMODE", "VEHICLE_CATEGORY", "ENERGYTYPEGROUP"]:
    sns.countplot(data=data_booking, x=col)
    if len(np.unique(data_booking[col].dropna())) > 2:
        plt.xticks(rotation=90)
    plt.show()

In [ ]:
print("Fraction of oneway", sum(data_booking["TRIPMODE"].str.contains("OneWay"))/len(data_booking))
print("Fraction of annulliert", sum(data_booking["RESERVATIONSTATE"].str.contains("annu"))/len(data_booking))
print("Fraction of Elektro", sum(data_booking["ENERGYTYPEGROUP"].dropna().str.contains("Electro"))/len(data_booking))

### Add the newly created columns

In [ ]:
data_booking["DURATION_HOURS"] = duration_sec
data_booking["SYS_CREATE_TS"] = sys_create_timestamp
data_booking["RESERVATION_FROM_TS"] = reservation_from_timestamp
data_booking["CREATED_BEFORE_HOURS"] = spontaneous

## Create stations table

In [ ]:
start_cols = ['BASESTART_NO', 'BASESTART_NAME', 'BASESTART_LAT', 'BASESTART_LON']
end_cols = ['BASEEND_NO', 'BASEEND_NAME','BASEEND_LAT', 'BASEEND_LON']

data_stations = data_booking[start_cols]
rename_dict = {c: c.split("_")[-1] for c in start_cols}
data_stations = data_stations.rename(columns=rename_dict)
add_end_stations =  data_booking[end_cols]
rename_dict = {c: c.split("_")[-1] for c in end_cols}
add_end_stations = add_end_stations.rename(columns=rename_dict)
data_stations = pd.concat([data_stations, add_end_stations])

In [ ]:
def most_often(x):
    uni, counts = np.unique(x, return_counts=True)
    return uni[np.argmax(counts)]

In [ ]:
stations_final = data_stations.groupby("NO").agg({"NAME": "first", "LAT": most_often, "LON": most_often})
stations_final

In [ ]:
assert len(stations_final) == len(np.unique(data_booking["BASESTART_NO"]))

In [ ]:
stations_final = lon_lat_to_geom(stations_final)
stations_final

In [ ]:
stations_final = stations_final.reset_index().rename(columns={"NO": "STATION_NO"}).set_index("STATION_NO")

#### WRITE 

In [ ]:
write_geodataframe(stations_final, "data_cleaned/station.csv")

In [ ]:
# read for test
read_geodataframe("data_cleaned/station.csv").set_index("STATION_NO")

#### Exploration for the following problem: Stations have varying longitude and latitude values

Check here how high the std is

In [ ]:
data_stations_cleaned = data_stations.groupby("NO").agg({"NAME": "first", "LAT": ["mean", "std"], "LON": ["mean", "std"]})
data_stations_cleaned


In [ ]:
std_too_high = data_stations_cleaned[data_stations_cleaned["LAT"]["std"]> 0.001].index
critical = data_stations.set_index("NO").loc[std_too_high]
critical.groupby("NO").agg({"LAT":"std"})

## Clean and save reservations table

#### Drop all columns that are part of the vehicle or of the stations table

In [ ]:
data_booking = data_booking.drop(columns=['BRAND_NAME','MODEL_NAME',"VEHICLE_CATEGORY",
                                          "BRAND_NAME", "ENERGYTYPE", 'BASESTART_NAME',
                                          'BASESTART_LAT', 'BASESTART_LON', 'BASEEND_NAME',
                                          'BASEEND_LAT', 'BASEEND_LON'])

#### Write data

In [ ]:
data_booking.set_index("RESERVATION_NO").to_csv("data_cleaned/booking.csv", index="RESERVATION_NO")

In [ ]:
# read for check
test = pd.read_csv("data_cleaned/booking.csv", index_col="RESERVATION_NO")
print(len(test), len(data_booking), len(data_booking_cleaned))
print(test.columns)

## Rename columns (convert all columns all lower case)

Loaded from data_cleaned folder and saved to the same, overwriting the previous cleaned files

In [ ]:
indices = ["VEHICLE_NO", "PERSON_NO", "STATION_NO", "RESERVATION_NO"]

for df_name, index_name in zip(["vehicle", "user", "station", "booking"], indices):

    # read
    if df_name == "booking" or df_name=="vehicle":
        df = pd.read_csv(f"data_cleaned/{df_name}.csv", index_col = index_name)
    else:
        df = read_geodataframe(f"data_cleaned/{df_name}.csv").set_index(index_name)
    
    # modify columns
    new_names = {name:name.lower() for name in df.reset_index().columns}
    if df_name=="vehicle":
        new_names["COUNT"] = "entries_count"
    elif df_name == "booking":
        new_names["BASESTART_NO"] = "start_station_no"
        new_names["BASEEND_NO"] = "end_station_no"
    df = df.reset_index().rename(columns=new_names).set_index(index_name.lower())
    
    # write
    if df_name == "booking" or df_name=="vehicle":
        df.to_csv(f"data_cleaned/{df_name}.csv", index=index_name.lower())
    else:
        write_geodataframe(df, f"data_cleaned/{df_name}.csv")

## Further processing of reservations:

### Split into several parts: Service reservations, cancelled, outliers and final clean reservations

In [ ]:
data_booking = pd.read_csv("data_cleaned/booking.csv", index_col="reservation_no")


#### 1) service reservations

In [ ]:
service_reservation = data_booking[data_booking["reservationtype"]!="Normal"]

In [ ]:
service_reservation.to_csv(os.path.join("data_cleaned", "service_reservation.csv"), index="reservation_no")

In [ ]:
# it not only includes service but also "BC-upload"
service_reservation["reservationtype"].unique()

In [ ]:
# reduce booking data to the rest
data_booking = data_booking[data_booking["reservationtype"]=="Normal"]

#### 2) Cancelled bookings

In [ ]:
len(data_booking), len(service_reservation)

In [ ]:
# There are rows where there is a canceldate, but reservationstate is not "annulliert" and everything else looks normal
# only 44 rows, so we just ignore that and delete it from the data
cond1 = ~pd.isna(data_booking["canceldate"])
cond2 = data_booking["reservationstate"]!= "annulliert"
data_booking[cond1 & cond2]

In [ ]:
cond_cancelled = ~pd.isna(data_booking["canceldate"]) | (data_booking["reservationstate"] == "sofortige Rückgabe") | (data_booking["reservationstate"] == "annulliert")
canceled_bookings = data_booking[cond_cancelled]
canceled_bookings.to_csv(os.path.join("data_cleaned", "cancelled_reservation.csv"), index="reservation_no")

In [ ]:
# reduce to rest
data_booking = data_booking[~cond_cancelled]

In [ ]:
# leftover: abgeschlossen, erfasst, geändert
np.unique(data_booking["reservationstate"].values, return_counts=True)

#### 3) TODO: outliers ( only bookings that are too long etc) --> currently not filtered out in reservation.csv
Open questions: bookings that are too short? free floating?

In [ ]:
# Outliers are the ones that start much earlier 
# TODO: outliers are the ones with more than 7 days of booking (168h) - delete or not??
# why not: because they might be relevant 
cond_outlier = data_booking["reservationfrom"] < "2019" # data_booking["duration_hours"] > 168
outlier_bookings = data_booking[cond_outlier]
outlier_bookings.to_csv(os.path.join("data_cleaned", "outlier_reservation.csv"), index="reservation_no")

In [ ]:
# CODE to check what columns contain zeros and what might be other outliers
data_booking.columns #[data_booking["duration_hours"]<.1]
# np.unique(data_booking["reservationtype"].values, return_counts=True)
for col in data_booking.columns:
    if any(pd.isna(data_booking[col])):
        print(col)
    #[pd.isna(data_booking)]
data_booking[pd.isna(data_booking["drive_km"])][["drive_km", "drive_firststart"]]

#### 4) save the leftover part

In [ ]:
data_booking = data_booking[~cond_outlier]
data_booking.to_csv(os.path.join("data_cleaned", "reservation.csv"), index="reservation_no")

## New data: Vehicle to base

In [19]:
path = "../../../data/V2G4Carsharing/20220204_eth_vehicle_to_base.xlsx"
v2b = pd.read_excel(path)

In [20]:
new_names = {name:name.lower() for name in v2b.reset_index().columns}
new_names["BASE_NO"] = "station_no"
v2b.rename(columns=new_names, inplace=True)

BASE_DATE = pd.to_datetime('2019-01-01 00:00:00.000')
FINAL_DATE = pd.to_datetime("2020-07-31 23:59:59.999")

In [21]:
v2b_in_range = v2b[(v2b["bizbeg"] <= FINAL_DATE) & (v2b["bizend"] > BASE_DATE)]

In [22]:
v2b_in_range.to_csv("../data_cleaned/vehicle_to_base.csv")

In [23]:
len(v2b_in_range)

23363

In [27]:
v2b_in_range.index.name = "v2b_id"

In [28]:
v2b_in_range.index.name.lower()

'v2b_id'

## Write to PostgreSQL database

### First run in pgAdmin:

CREATE EXTENSION postgis;


In [ ]:
import psycopg2

In [ ]:
import json
with open("../../../goeco_login.json", "r") as infile:
    db_credentials = json.load(infile)

In [ ]:
from sqlalchemy import create_engine
def get_con():
    return psycopg2.connect(**db_credentials)

### 1) Vehicles

In [ ]:
vehicles = pd.read_csv("data_cleaned/vehicle.csv", index_col="vehicle_no")
vehicles

In [ ]:
engine = create_engine('postgresql+psycopg2://', creator=get_con)
vehicles.to_sql("vehicle", engine, schema="mobility", index=True, index_label="vehicle_no", chunksize=10000)
conn.close()

### 2) Users

In [ ]:
user = read_geodataframe("data_cleaned/user.csv").set_index("person_no")
user

In [ ]:
# create_table_query =  "CREATE TABLE IF NOT EXISTS mobility.user\
#     (person_no SERIAL PRIMARY KEY, agegroup VARCHAR, gender CHAR,\
#     lat FLOAT, lon FLOAT, language VARCHAR, abogroup VARCHAR, firstentrydate DATETIME,\
#     age_group_int INT, firstentry_timestamp BIGINT)"
# execute_query(create_table_query)

#  dtype = {"person_no": "SERIAL PRIMARY KEY", 
#       "agegroup": "VARCHAR",
#       "gender" : "CHAR",
#       "lat" : "FLOAT",
#       "lon" : "FLOAT",
#       "language" : "VARCHAR",
#       "abogroup" : "VARCHAR",
#       "firstentrydate" : "DATETIME",
#       "agegroup_int" : "INT",
#       "firstentry_timestamp" : "BIGINT"
#          }         

In [ ]:
engine = create_engine('postgresql+psycopg2://', creator=get_con)
user.to_postgis("user", engine, schema="mobility", index=True, index_label="person_no", chunksize=10000, dtype=None)


#### There are a lot of warnings, the reason is simply that geom is NaN for some users

In [ ]:
user[pd.isna(user["lat"])]

### 3) Stations

In [ ]:
station = read_geodataframe("data_cleaned/station.csv").set_index("station_no")
station

In [ ]:
# conn = psycopg2.connect(**db_credentials)
# dtype = {"station_no": "SERIAL PRIMARY KEY", 
#           "name": "VARCHAR",
#           "lat" : "FLOAT",
#           "lon" : "FLOAT",
#           "geom": "POINT"
#           https://stackoverflow.com/questions/38361336/write-geodataframe-into-sql-database
#          }
engine = create_engine('postgresql+psycopg2://', creator=get_con)
station.to_postgis("station", engine, schema="mobility", index=True, index_label="station_no", chunksize=10000)
# conn.close()

### 4) service reservations (relocations)

In [ ]:
service_reservation = pd.read_csv("data_cleaned/service_reservation.csv").set_index("reservation_no")

In [ ]:
engine = create_engine('postgresql+psycopg2://', creator=get_con)
service_reservation.to_sql("service_reservation", engine, schema="mobility", index=True, index_label="reservation_no", chunksize=10000)


### 5) cancelled reservations

In [ ]:
cancelled_reservation = pd.read_csv("data_cleaned/cancelled_reservation.csv").set_index("reservation_no")

In [ ]:
engine = create_engine('postgresql+psycopg2://', creator=get_con)
cancelled_reservation.to_sql("cancelled_reservation", engine, schema="mobility", index=True, index_label="reservation_no", chunksize=10000)


### 6) cleaned reservations

In [ ]:
reservation = pd.read_csv("data_cleaned/reservation.csv").set_index("reservation_no")

In [ ]:
engine = create_engine('postgresql+psycopg2://', creator=get_con)
reservation.to_sql("reservations", engine, schema="mobility", index=True, if_exists = 'replace', index_label="reservation_no", chunksize=10000)


### OLD VERSION: all bookings at once

In [ ]:
booking = pd.read_csv("data_cleaned/booking.csv").set_index("reservation_no")

In [ ]:
booking

In [ ]:
engine = create_engine('postgresql+psycopg2://', creator=get_con)
booking.to_sql("reservation", engine, schema="mobility", index=True, index_label="reservation_no", chunksize=10000)